<a href="https://colab.research.google.com/github/AlexKressner/WS25_Supply_Chain_Optimierung/blob/main/Raffinerieproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# pip als Paketmanager
! pip install -q pyscipopt

In [42]:
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Produktionsplanung

In [43]:
# Erstellen einer Modellinstanz
scip = Model()

## Indexmengen

In [44]:
J = ['K1','K2','K3','K4'] # Menge der Produkte
I = ['R1','R2','R3','Raffineriekapazität'] # Menge der Produktionsfaktoren

## Parameter (Daten)

In [45]:
c={} # Deckungsbeiträge
c['K1']=5
c['K2']=8
c['K3']=15
c['K4']=3

In [46]:
a={} # Produktionskoeffizienten, a[i,j]
a['R1']={'K1':1,'K2':1,'K3':2,'K4':1}
a['R2']={'K1':2,'K2':1,'K3':3,'K4':0}
a['R3']={'K1':0,'K2':1,'K3':2,'K4':1}
a['Raffineriekapazität']={'K1':1,'K2':1,'K3':1,'K4':1}

In [47]:
b={} # Kapazitäten Produktionsfaktoren
b['R1']=1000
b['R2']=500
b['R3']=1500
b['Raffineriekapazität']=2000

## Entscheidungsvariablen

In [48]:
# Definition der Entscheidungsvariablen
x={}
for j in J:
    x[j] = scip.addVar(vtype='C', lb=0, ub=None, name=j)
print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [K1, K2, K3, K4]


## Zielfunktion

In [49]:
# Maximierung der gesamten Deckungsbeiträge
scip.setObjective(quicksum(c[j]*x[j] for j in J), sense="maximize")

## Nebenbedingungen/ Restriktionen

In [50]:
for i in I:
    scip.addCons(quicksum(a[i][j]*x[j] for j in J)<=b[i], name=i)

In [51]:
print('Nebenbedingungen =', scip.getConss())

Nebenbedingungen = [R1, R2, R3, Raffineriekapazität]


## Berechnung der Lösung

In [52]:
scip.optimize()
# Status des Solvers
status = scip.getStatus()
gap = scip.getGap()
print(f"Status des Solvers: {status} , GAP={gap}% \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', scip.getObjVal())
    for j in J:
        print(f'{j} =', round(scip.getVal(x[j])))
else:
    print('Problem hat keine Lösung')

Status des Solvers: optimal , GAP=0.0% 

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 5500.0
K1 = 0
K2 = 500
K3 = 0
K4 = 500
